In [1]:
from config import model_api

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
import requests

In [169]:
ticker = str(input("Enter Company Ticker")).upper()
incomeAnnual_url = f"https://financialmodelingprep.com/api/v3/income-statement/{ticker}?apikey={model_api}"
# get income statement data
incomeQuarter_url = f"https://financialmodelingprep.com/api/v3/income-statement/{ticker}?apikey={model_api}&period=quarter&limit=100"

# get cash flow statement data
cashFlowQuarter_url = f"https://financialmodelingprep.com/api/v3/cash-flow-statement/{ticker}?apikey={model_api}&period=quarter&limit=100"

# income statement growth numbers
incomeGrowthQuarter_url = f"https://financialmodelingprep.com/api/v3/income-statement-growth/{ticker}?limit=100&period=quarter&apikey={model_api}"

# key stats investors look for
keyStats_url = f"https://financialmodelingprep.com/api/v3/profile/{ticker}?apikey={model_api}"

# key metrics
keyMetrics_url = f"https://financialmodelingprep.com/api/v3/key-metrics/{ticker}?period=quarter&limit=100&apikey={model_api}"

Enter Company Tickerfd.v


In [170]:
ticker

'FD.V'

In [171]:
income_json = requests.get(incomeQuarter_url).json()
cashFlow_json = requests.get(cashFlowQuarter_url).json()
keyMetrics_json = requests.get(keyMetrics_url).json()

In [157]:
keyStats_json = requests.get(keyStats_url).json()

In [163]:
len(keyMetrics_json)

5

In [42]:
# request for income growth data
incomeGrowth_json = requests.get(incomeGrowthQuarter_url).json()

In [172]:
income_json

[]

In [166]:
len(income_json)

0

In [144]:
# create a funciton that outputs all financial statement information
def createIncomeStatement(incomeStatement_file):
    """
    THIS FUNCTION CREATES A DATAFRAME FROM INCOME STATEMENT DATA
    """
    for i in range(len(incomeStatement_file)):
        # define the rows for the DataFrame
        rows = ["Revenue",
                "Revenue Growth",
                "Cost of Revenue",
                "Gross Profit",
                'Operating Margin', 
                "Operating Income/Loss", 
                "Operating Income/Loss Growth",                  
                "Net Income",
               "Net Margin"]

        # determine revenue
        revenue = incomeStatement_file[i]['revenue'] / 1000
        # determine revenue growth
        try:
            revenuePreviousQuarter = incomeStatement_file[i+1]['revenue'] / 1000
            revenueGrowth = (revenue - revenuePreviousQuarter) / revenuePreviousQuarter
        except IndexError:
            revenueGrowth = np.nan
        except ZeroDivisionError:
            revenueGrowth = np.nan
        # determine cost of revenue
        costRevenue = incomeStatement_file[i]['costOfRevenue'] / 1000
        # determine gross profit
        gross_profit = incomeStatement_file[i]['grossProfit'] / 1000
        # determine operating expenses
        operating_expenses = incomeStatement_file[i]['operatingExpenses'] / 1000
        # determine net income
        net_income = incomeStatement_file[i]['netIncome'] / 1000
        # calculate operating income
        operating_income = gross_profit - operating_expenses
        # operating income growth
        try:
            grossProfitPreviousQuarter = incomeStatement_file[i+1]['grossProfit'] / 1000
            operatingExpensesPreviousQuarter = incomeStatement_file[i+1]['operatingExpenses'] / 1000
            operatingIncomePreviousQuarter = grossProfitPreviousQuarter - operatingExpensesPreviousQuarter
            operating_income_growth = (operating_income - operatingIncomePreviousQuarter) / operatingIncomePreviousQuarter
        except IndexError:
            operating_income_growth = np.nan
        except ZeroDivisionError:
            operating_income_growth = np.nan
        
        # calculate operating margin if revenue is greater than 0
        try:
            operating_margin = operating_income / revenue
        except ZeroDivisionError:
            operating_margin = np.nan
            
        # Calculate net margin
        netMargin = net_income / revenue
        
        
        # create a dataframe on the first iteration and add new columns on the rest 
        if i == 0:
            df = pd.DataFrame({incomeStatement_file[i]['date']: [revenue,
                                                                 revenueGrowth,
                                                                 costRevenue,
                                                                 gross_profit,
                                                                 operating_margin, 
                                                                 operating_income,
                                                                 operating_income_growth,                                                               
                                                                 net_income,
                                                                 netMargin]}, 
                                                         index=rows)
        else:
            df[incomeStatement_file[i]['date']] = [revenue,
                                                   revenueGrowth,
                                                   costRevenue,
                                                   gross_profit,
                                                   operating_margin, 
                                                   operating_income,
                                                   operating_income_growth,                                                   
                                                   net_income,
                                                   netMargin]
    # display company ticker as index title
    df.index.name = incomeStatement_file[0]['symbol']

    return df

In [145]:
createIncomeStatement(income_json)

,2020-09-30,2020-06-30,2020-03-31,2019-12-31,2019-09-30
BNGO,,,,,
Revenue,2196.000000,1182.000000,1136.000000,2789.130000,3312.997000
Revenue Growth,0.857868,0.040493,-0.592705,-0.158125,NaN
Cost of Revenue,1460.000000,603.000000,856.000000,1690.759000,2375.100000
Gross Profit,736.000000,579.000000,280.000000,1098.371000,937.897000
Operating Margin,-4.657104,-6.290186,-8.593310,-2.567285,-1.715998
Operating Income/Loss,-10227.000000,-7435.000000,-9762.000000,-7160.493000,-5685.096000
Operating Income/Loss Growth,0.375521,-0.238373,0.363314,0.259520,NaN
Net Income,-10792.000000,-8074.000000,-10510.000000,-7900.595000,-6398.166000
Net Margin,-4.914390,-6.830795,-9.251761,-2.832638,-1.931232


In [22]:
cashFlow_json

[{'date': '2020-09-30',
  'symbol': 'AYRO',
  'reportedCurrency': 'USD',
  'fillingDate': '2020-11-06',
  'acceptedDate': '2020-11-06 08:00:37',
  'period': 'Q3',
  'netIncome': -2680726.0,
  'depreciationAndAmortization': 146177.0,
  'deferredIncomeTax': 0.0,
  'stockBasedCompensation': 167767.0,
  'changeInWorkingCapital': -1742486.0,
  'accountsReceivables': -105307.0,
  'inventory': -466128.0,
  'accountsPayables': 226716.0,
  'otherWorkingCapital': 58610.0,
  'otherNonCashItems': 66659.0,
  'netCashProvidedByOperatingActivities': -3824572.0,
  'investmentsInPropertyPlantAndEquipment': 0.0,
  'acquisitionsNet': 0.0,
  'purchasesOfInvestments': 0.0,
  'salesMaturitiesOfInvestments': 0.0,
  'otherInvestingActivites': 0.0,
  'netCashUsedForInvestingActivites': 0.0,
  'debtRepayment': -639483.0,
  'commonStockIssued': 22335003,
  'commonStockRepurchased': 0.0,
  'dividendsPaid': 0.0,
  'otherFinancingActivites': 0.0,
  'netCashUsedProvidedByFinancingActivities': 0.0,
  'effectOfForexCh

In [113]:
## CASH FLOW STATEMENT

# Create a function that creates the cash flow statement
def createCashFlowStatement(cashFlow_file):
    for num in range(len(cashFlow_file)):
        cashRows = ["cash from operating activities",
                   "cash used for investing activities",
                   "cash used provided by financing activities",
                   "acquisitions",
                   "purchase of investments",
                   "sales maturities of investments",
                   "net change in cash",
                   "free cash flow",
                   "common stock repurchased"]
        cashOperatingActivities = cashFlow_file[num]['netCashProvidedByOperatingActivities']
        cashInvestingActivities = cashFlow_file[num]['netCashUsedForInvestingActivites']
        cashFinancingActivities = cashFlow_file[num]['netCashUsedProvidedByFinancingActivities']
        acquisitions = cashFlow_file[num]['acquisitionsNet']
        purchaseInvestments = cashFlow_file[num]['purchasesOfInvestments']
        investmentMaturities = cashFlow_file[num]['salesMaturitiesOfInvestments']
        changeCash = cashFlow_file[num]['netChangeInCash']
        freeCashFlow = cashFlow_file[num]['freeCashFlow']
        stockRepurchase = cashFlow_file[num]['commonStockRepurchased']
        # Debt repayments
        debtRepayments = 

        # create a dataframe on the first iteration and add new columns on the rest 
        if num == 0: 
            df = pd.DataFrame({cashFlow_file[num]['date']: [cashOperatingActivities,
                                                cashInvestingActivities,
                                                cashFinancingActivities,
                                                acquisitions,
                                                purchaseInvestments,
                                                investmentMaturities,
                                                changeCash,
                                                freeCashFlow,
                                                stockRepurchase]}, 
                                             index=cashRows)
        else:
            df[cashFlow_file[num]['date']] = [cashOperatingActivities,
                                                cashInvestingActivities,
                                                cashFinancingActivities,
                                                acquisitions,
                                                purchaseInvestments,
                                                investmentMaturities,
                                                changeCash,
                                                freeCashFlow,
                                                stockRepurchase]
            
    # display company ticker as index title
    df.index.name = cashFlow_file[0]['symbol']
    
    return df

In [114]:
createCashFlowStatement(cashFlow_json)

,2020-09-30,2020-06-30,2020-03-31,2019-12-31,2019-09-30
AYRO,,,,,
cash from operating activities,-3824572.0,-2040300.0,-677620.0,-195960.0,-508400.0
cash used for investing activities,0.0,2838370.0,-30080.0,-24270.0,0.0
cash used provided by financing activities,0.0,7185930.0,0.0,2300000.0,0.0
acquisitions,0.0,0.0,0.0,0.0,0.0
purchase of investments,0.0,0.0,0.0,0.0,0.0
sales maturities of investments,0.0,0.0,0.0,0.0,0.0
net change in cash,19998718.0,7984000.0,-707700.0,2079770.0,-168336.0
free cash flow,-4164991.0,-2292750.0,-677620.0,-195960.0,-537088.0
common stock repurchased,0.0,6971330.0,0.0,2515000.0,0.0


In [29]:
pd.DataFrame({cashFlow_json[0]['date']: [cashOperatingActivities,
                                        cashInvestingActivities,
                                        cashFinancingActivities,
                                        acquisitions,
                                        purchaseInvestments,
                                        investmentMaturities,
                                        changeCash,
                                        freeCashFlow,
                                        stockRepurchase]}, 
             index=cashRows)

,2020-09-30
cash from operating activities,-3824572.0
cash used for investing activities,0.0
cash used provided by financing activities,0.0
acquisitions,0.0
purchase of investments,0.0
sales maturities of investments,0.0
net change in cash,19998718.0
free cash flow,-4164991.0
common stock repurchased,0.0


In [16]:
test = requests.get(keyStats_url)

In [22]:
test2 = requests.get(incomeStatement_url)

In [25]:
test2.json()[0]

{'date': '2020-09-26',
 'symbol': 'AAPL',
 'reportedCurrency': 'USD',
 'fillingDate': '2020-10-30',
 'acceptedDate': '2020-10-29 18:06:25',
 'period': 'FY',
 'revenue': 274515000000,
 'costOfRevenue': 169559000000,
 'grossProfit': 104956000000,
 'grossProfitRatio': 0.382332477278109,
 'researchAndDevelopmentExpenses': 18752000000,
 'generalAndAdministrativeExpenses': 19916000000,
 'sellingAndMarketingExpenses': 0.0,
 'otherExpenses': -87000000,
 'operatingExpenses': 38668000000,
 'costAndExpenses': 208227000000,
 'interestExpense': 2873000000,
 'depreciationAndAmortization': 11056000000,
 'ebitda': 81020000000,
 'ebitdaratio': 0.295138699160337,
 'operatingIncome': 66288000000,
 'operatingIncomeRatio': 0.244398302460703,
 'totalOtherIncomeExpensesNet': -87000000,
 'incomeBeforeTax': 67091000000,
 'incomeBeforeTaxRatio': 0.244398302460703,
 'incomeTaxExpense': 9680000000,
 'netIncome': 57411000000,
 'netIncomeRatio': 0.209136112780722,
 'eps': 3.36,
 'epsdiluted': 3.36,
 'weightedAverag